In [5]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi

import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib import pyplot as plt

# Indices correct?

compare indices for manually created small atomic kernel with full kernel

In [2]:
data, molecule_size = qmi.load_alchemy_data(qmi.wrapper_alch_data())

In [3]:
# choose random molecules
global_tr, global_test = qmi.get_indices(len(molecule_size), 3)
global_tr = np.array([ 14, 519, 549])
tr_ind = qmi.get_local_idx(global_tr, molecule_size)
test_ind = qmi.get_local_idx(global_test, molecule_size)

In [ ]:
# calculate representation and atomic kernels
reps = []
for i in global_tr:
    rep_fchl = qml.fchl.generate_representation(data[i][:, 1:4], data[i][:,0], max_size=molecule_size[i])
    reps.append(rep_fchl)

In [ ]:
sigma = 432.8761281083057
kernel00 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[0], kernel_args={"sigma":[sigma],})[0]
kernel01 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[1], kernel_args={"sigma":[sigma],})[0]
kernel02 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[2], kernel_args={"sigma":[sigma],})[0]
kernel11 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[1], reps[1], kernel_args={"sigma":[sigma],})[0]
kernel12 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[1], reps[2], kernel_args={"sigma":[sigma],})[0]
kernel22 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[2], reps[2], kernel_args={"sigma":[sigma],})[0]

In [ ]:
kernel3x3 = np.block([[kernel00, kernel01, kernel02], [kernel01.T, kernel11, kernel12], [kernel02.T, kernel12.T, kernel22]])

In [ ]:
# kernel between molecule at index 0 and molecule at index 2

# select correct indices
start_idx = molecule_size[14] + molecule_size[519]
print(start_idx)
end_idx = start_idx + molecule_size[549] - 1
print(end_idx)

# pick slice
kernel3x3_02 = kernel3x3[0:16, 26:39]

In [ ]:
# load kernel

In [ ]:
kernel_full = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig432.8761281083057')

In [ ]:
# indices in full kernel
start_idx_full_row = molecule_size[0:14].sum()
end_idx_full_row = start_idx_full_row + molecule_size[14]-1

In [ ]:
start_idx_full_column = molecule_size[0:549].sum()
end_idx_full_column= start_idx_full_column + molecule_size[549]-1

In [ ]:
# select correct indices
kernel_full_slice = kernel_full[start_idx_full_row:end_idx_full_row+1, start_idx_full_column:end_idx_full_column+1]

In [ ]:
kernel02 - kernel_full_slice

In [ ]:
kernel3x3_02 - kernel_full_slice

## indices are correctly assigned in full matrix
## what about kernel splitting during hyperparameter splitting?

In [ ]:
print(start_idx_full_row)
print(end_idx_full_row)

In [ ]:
# split kernel
tr_kernel, test_kernel = qmi.split_kernel(kernel_full, tr_ind, test_ind)

In [ ]:
# correct kernel selected? yes identical to manually computed kernel
print(np.amax(tr_kernel-kernel3x3))
print(np.amin(tr_kernel-kernel3x3))

## Difference between kernels with different sigma

In [15]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi

In [27]:
kernel_full1 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig432.8761281083057')

In [30]:
kernel_full2 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig10000.0')

In [31]:
kernel_full2-kernel_full1

array([[0.00000000e+00, 1.43517254e-08, 1.11022302e-16, ...,
        5.08949919e-07, 3.33393885e-06, 3.33395256e-06],
       [1.43517254e-08, 0.00000000e+00, 1.43518569e-08, ...,
        4.97603471e-07, 3.32422847e-06, 3.32425100e-06],
       [1.11022302e-16, 1.43518569e-08, 0.00000000e+00, ...,
        5.08950024e-07, 3.33393894e-06, 3.33395265e-06],
       ...,
       [5.08949919e-07, 4.97603471e-07, 5.08950024e-07, ...,
        0.00000000e+00, 4.31655907e-06, 4.31658268e-06],
       [3.33393885e-06, 3.32422847e-06, 3.33393894e-06, ...,
        4.31655907e-06, 0.00000000e+00, 4.35651515e-13],
       [3.33395256e-06, 3.32425100e-06, 3.33395265e-06, ...,
        4.31658268e-06, 4.35651515e-13, 0.00000000e+00]])

## Training error as function of sigma

In [52]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi
import glob
import re

# paths to kernels
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig*')

# indices for training kernel selection
global_tr = np.array([ 14, 519, 549])
data, molecule_size = qmi.load_alchemy_data(qmi.wrapper_alch_data())
tr_ind = qmi.get_local_idx(global_tr, molecule_size)

global_tr, global_test = qmi.get_indices(len(molecule_size), 3)
global_tr = np.array([ 14, 519, 549])
tr_ind = qmi.get_local_idx(global_tr, molecule_size)
test_ind = qmi.get_local_idx(global_test, molecule_size)

error = []
kernel_diff= []

for p in paths:
    if re.search('lam',p):
        pass
    else:
        kernel_full1 = np.loadtxt(p)
        
        # calculate training error
        tr_kernel, test_kernel = qmi.split_kernel(kernel_full1, tr_ind, test_ind)
        labels = qmi.generate_label_vector(data, molecule_size.sum(), value='atomisation')
        tr_label = labels[tr_ind]

        coeffs = qml.math.cho_solve(tr_kernel, tr_label)
        pred_label = np.dot(tr_kernel, coeffs)
        tr_error = np.abs(tr_label-pred_label)
        
        error.append([float(p.split('sig')[1]), tr_error.mean()])
        
        # maximum difference between to elements in training kernel
        max_diff = np.amax(tr_kernel) - np.amin(tr_kernel)
        kernel_diff.append([float(p.split('sig')[1]), max_diff])

In [58]:
error.sort()
error
tr_errors = np.array(error)

In [59]:
kernel_diff.sort()
kernel_diff_np = np.array(kernel_diff)
kernel_diff_np

array([[1.00000000e-01, 1.00000000e+00],
       [2.84803587e-01, 9.99988513e-01],
       [8.11130831e-01, 7.53962628e-01],
       [2.31012970e+00, 1.58760287e-01],
       [6.57933225e+00, 2.10877620e-02],
       [1.87381742e+01, 2.62415228e-03],
       [5.33669923e+01, 3.23890518e-04],
       [1.51991108e+02, 3.99364074e-05],
       [4.32876128e+02, 4.92363316e-06],
       [1.23284674e+03, 6.07009819e-07],
       [3.51119173e+03, 7.48350275e-08],
       [1.00000000e+04, 9.22601229e-09]])

In [60]:
# tr_errors = np.array([[0.1, 9.533845087234126e-07],
#  [0.2848035868435802, 1.1146930695175114e-05],
#  [0.8111308307896871, 0.0002756277498121068],
#  [2.3101297000831593, 0.0035897837548015087],
#  [6.579332246575679, 3273337966246805.5],
#  [18.73817422860383, 150713133386679.62],
#  [53.36699231206307, 4.2190757635430615e+18],
#  [151.99110829529332, 1.9391011745425462e+21],
#  [432.8761281083057, 1.3728786113167146e+22],
#  [1232.8467394420659, 5.937093237082583e+22],
#  [3511.1917342151273, 1.8727704323267042e+27],
#  [10000.0, 3.2333931384220235e+23]])

In [64]:
plt.plot(tr_errors[:,0], tr_errors[:,1], '-o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\sigma$')
plt.ylabel('training error')

Text(0, 0.5, 'training error')

In [63]:
plt.plot(kernel_diff_np[:,0], kernel_diff_np[:,1], '-o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\sigma$')
plt.ylabel('Max Kerneldiff')

Text(0, 0.5, 'Max Kerneldiff')